In [ ]:
import numpy as np

class BinaryLogisticRegression:
    def __init__(self, n_features, batch_size, conv_threshold):
        self.n_features = n_features
        self.weights = np.zeros(n_features + 1)  # extra element for bias
        self.alpha = 0.01
        self.batch_size = batch_size
        self.conv_threshold = conv_threshold

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def train(self, X, Y):
        # intializing values
        converge = False
        epochs = 0
        n_examples = X.shape[0]

        while not converge:
            # update # of epochs
            epochs += 1
            # acquire indices for shuffling of X and Y
            indices = np.arange(n_examples)
            np.random.shuffle(indices)
            X = X[indices]
            Y = Y[indices]
            # calc last epoch loss
            last_epoch_loss = self.loss(X, Y)
            # for the # of batches
            for i in range(0, n_examples, self.batch_size):
                X_batch = X[i:i + self.batch_size]
                Y_batch = Y[i:i + self.batch_size]
                # reinitialize gradient to be 0s
                grad = np.zeros(self.weights.shape)
                # for each pair in the batch
                for x, y in zip(X_batch, Y_batch):
                    prediction = self.sigmoid(np.dot(self.weights, x))
                    # gradient calculation
                    error = prediction - y
                    grad += error * x
                # update weights
                self.weights -= (self.alpha * grad) / self.batch_size
            epoch_loss = self.loss(X, Y)
            if abs(epoch_loss - last_epoch_loss) < self.conv_threshold:
                converge = True
        return epochs

    def loss(self, X, Y):
        n_examples = X.shape[0]
        total_loss = 0
    
        for i in range(n_examples):
            linear_output = np.dot(self.weights, X[i])
            y = 1 if Y[i] == 1 else -1  # Convert labels to {-1, 1}
            # compute logistic loss
            logistic_loss = np.log(1 + np.exp(-y * linear_output))
            total_loss += logistic_loss
    
        return total_loss / n_examples

    
    def predict(self, X):
        # multiply X by weights of model
        predictions = self.sigmoid(X @ self.weights)
        return np.where(predictions >= 0.5, 1, 0)

    def accuracy(self, X, Y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == Y)
        return accuracy


In [ ]:
class AllPairsLogisticRegression:
    def __init__(self, n_classes, binary_classifier_class, n_features, batch_size, conv_threshold):
        """
        Initialize the all-pairs logistic regression model.
        @param n_classes: Number of classes in the dataset, an integer.
        @param binary_classifier_class: Class for binary logistic regression, a class object.
        @param n_features: Number of features in the dataset, an integer.
        @param batch_size: Batch size for training the binary classifiers, an integer.
        @param conv_threshold: Convergence threshold for training, a float.
        @return: None
        """
        self.n_classes = n_classes
        self.classifiers = {} 
        self.n_features = n_features
        self.batch_size = batch_size
        self.conv_threshold = conv_threshold
        self.binary_classifier_class = binary_classifier_class

    def train(self, X, Y):
        """
        Train the all-pairs logistic regression model by training binary classifiers
        for each pair of classes in the dataset.
        @param X: Input features of the dataset, a numpy array of shape (n_samples, n_features).
        @param Y: Labels of the dataset, a numpy array of shape (n_samples,).
        @return: None
        """
        for class_i in range(self.n_classes):
            for class_j in range(class_i + 1, self.n_classes):
                SX = []
                SY = []
                for t in range(len(Y)):
                    if Y[t] == class_i:
                        SX.append(X[t])
                        SY.append(1)
                    elif Y[t] == class_j:
                        SX.append(X[t])
                        SY.append(-1)
                SX = np.array(SX)
                SY = np.array(SY)
                classifier = self.binary_classifier_class(
                    n_features=self.n_features,
                    batch_size=self.batch_size,
                    conv_threshold=self.conv_threshold
                )
                classifier.train(SX, SY)
                self.classifiers[(class_i, class_j)] = classifier

    def predict(self, X):
        """
        Predict the class labels for the input data using the trained classifiers.
        @param X: Input features to classify, a numpy array of shape (n_samples, n_features).
        @return: Predicted class labels, a numpy array of shape (n_samples,).
        """
        n_samples = X.shape[0]
        votes = np.zeros((n_samples, self.n_classes), dtype=int)
        for (class_i, class_j), classifier in self.classifiers.items():
            predictions = classifier.predict(X)
            votes[:, class_i] += (predictions == 1)
            votes[:, class_j] += (predictions == 0)
        return np.argmax(votes, axis=1)

    def accuracy(self, X, Y):
        """
        Calculate the accuracy of the model on the input data and labels.
        @param X: Input features of the dataset, a numpy array of shape (n_samples, n_features).
        @param Y: True labels of the dataset, a numpy array of shape (n_samples,).
        @return: Accuracy of the model as a float between 0 and 1.
        """
        predictions = self.predict(X)
        correct_predictions = np.sum(predictions == Y)
        return correct_predictions / len(Y)